In [ ]:
from IPython.display import clear_output,display, update_display
import time
import psycopg2
from psycopg2 import sql
from pybit.unified_trading import HTTP
from datetime import datetime, timezone
from Sdata import bybitkey,bybit_Secret
# init bybit session
session = HTTP(
    testnet=False,
    api_key=bybitkey,
    api_secret=bybit_Secret
)
# 資料庫連接參數
db_params = {
    "host": "123.241.217.251",
    "database": "mydatabase",
    "user": "myuser",
    "password": "mysecretpassword",
    "port": "5432"
}
trad_types = ['Buy','Sell']
# 連接資料庫
def connect_to_db():
    try:
        conn = psycopg2.connect(**db_params)
        return conn
    except (Exception, psycopg2.Error) as error:
        print("連接資料庫時出錯:", error)
        return None

# 執行查詢
def execute_query(conn, query, params=None):
    try:
        with conn.cursor() as cur:
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("執行時出錯:", error)
symbol = "BTCUSDT"
min_qty = 0.001  # 最小下單數量 根據交易對調整
print('init processing...')
output = display("start trading...", display_id=True)

def print_with_clear(message):
    clear_output(wait=True)
    print(message)

def close_all_position(trad_type):
    positions = session.get_positions(category="linear", symbol="BTCUSDT")
    Idx = 1
    if trad_type == 'Sell':
        Idx = 2
    if trad_type == 'Buy':
        trad_type = 'Sell'
    else:
        trad_type = 'Buy'
    response = session.place_order(
    category="linear",  # 或 "inverse"，根據合約類型設定
    symbol="BTCUSDT",
    side=trad_type,  # 平掉多單的方向為 "Sell"
    orderType="Market",
    qty=positions['result']['list'][0]['size'],
    reduceOnly=True,  # 設定為 True 確保這是一個平倉訂單
    timeInForce="GTC",  # Good Till Cancel
    positionIdx=Idx,
    )
    return response

def place_order(side, qty):
    try:
        position_idx = 1 if side == "Buy" else 2  #做多用 1 做空用 2
        order = session.place_order(
            category="linear",
            symbol=symbol,
            side=side,
            orderType="Market",
            qty=qty,
            positionIdx=position_idx
        )
        print(f"下單成功: {side} {qty} {symbol}")
        return order
    except Exception as e:
        print(f"下單失敗: {e}")
        return None
def get_position():
    try:
        position = session.get_positions(category="linear", symbol=symbol)
        return position['result']['list'][0]
    except Exception as e:
        print(f"獲取持倉資訊失敗: {e}")
        return None

def calculate_roi(position,trad_type):
    mode = 1
    if trad_type == 'Buy':
        mode = -1
    entry_price = float(position['avgPrice'])
    current_price = float(position['markPrice'])
    return (entry_price - current_price) / entry_price * 100 *mode #*-1代表作多

def trading_strategy(trad_type):
    while True:
        position = get_position()
        if position is None:
            continue

        size = float(position['size'])
        
        if size == 0:
            # if not order，start treading (small_order)
            place_order(trad_type, min_qty)
            input_db_trading_data()
        else:
            roi = calculate_roi(position,trad_type) * 100
            # print(f'ROI: {roi} %')
            # print_with_clear(f'ROI: {roi} %')
            update_display(f'ROI: {roi} % time=>{datetime.now()}', display_id=output.display_id)
            time.sleep(1)
            if roi >= 20:
                # ROI == 20%，平倉重新開始
                # place_order("Buy", size)
                # print('Order Buy')
                # time.sleep(1)  # 等待 order 執行
                # place_order("Sell", min_qty)
                # print('Order Sell')
                ans = close_all_position(trad_type)
                print('平倉',ans)
            elif roi <= -75:
                print(f'Order puls {trad_type}',datetime.now())
                # ROI == -50%，加倉
                place_order(trad_type, min_qty)
                input_db_trading_data()
    
        # time.sleep(5)  # 每 5 秒更新 output
def input_db_trading_data():
    conn = connect_to_db()
    positions = session.get_positions(category="linear", symbol="BTCUSDT")

    symbol= positions['result']['list'][0]['symbol']
    PositionIM= positions['result']['list'][0]['positionIM'] 
    Leverage= positions['result']['list'][0]['leverage']
    markPrice= positions['result']['list'][0]['markPrice'] 
    AvgPrice= positions['result']['list'][0]['avgPrice'] 
    Size= positions['result']['list'][0]['size']
    Side= positions['result']['list'][0]['side']
    CurRealisedPnl= positions['result']['list'][0]['curRealisedPnl'] 
    PositionValue= positions['result']['list'][0]['positionValue'] 
    createdTime= datetime.fromtimestamp(int(positions['result']['list'][0]['updatedTime']) / 1000, timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S')
    Time= datetime.fromtimestamp(int(positions['time']) / 1000, timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S')
    if conn:
        insert_query = sql.SQL("""
        INSERT INTO sptd.tdata 
        (symbol, asset, leverage, buy_price, sell_price, quantity, transaction_type, CurRealisedPnl, PositionValue, create_transaction_time, transaction_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """)
        
        params = (
            symbol, PositionIM, Leverage, markPrice, AvgPrice, Size, Side, 
            CurRealisedPnl, PositionValue, createdTime, Time
        )
        
        execute_query(conn, insert_query, params)

        conn.close()
#trad_type => 0:Buy 1:Sell
if __name__ == "__main__":
    trading_strategy(trad_types[1])

init processing...


'ROI: 18.738316044133704 % time=>2024-10-19 19:58:49.737969'

下單成功: Sell 0.001 BTCUSDT
平倉 {'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '1d2edbaa-1ef9-4283-84d6-438071bc7b37', 'orderLinkId': ''}, 'retExtInfo': {}, 'time': 1729252284315}
下單成功: Sell 0.001 BTCUSDT
Order puls Sell 2024-10-18 22:00:13.428675
下單成功: Sell 0.001 BTCUSDT
Order puls Sell 2024-10-18 22:09:38.817832
下單成功: Sell 0.001 BTCUSDT
平倉 {'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': 'b464baf0-82ff-41aa-8af8-780da3a15bc3', 'orderLinkId': ''}, 'retExtInfo': {}, 'time': 1729260924046}
下單成功: Sell 0.001 BTCUSDT
Order puls Sell 2024-10-18 22:34:00.792055
下單成功: Sell 0.001 BTCUSDT
Order puls Sell 2024-10-18 23:30:56.592614
下單成功: Sell 0.001 BTCUSDT
Order puls Sell 2024-10-19 02:37:10.839408
下單成功: Sell 0.001 BTCUSDT
平倉 {'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '6382a8a6-6c00-46dc-88e1-45af10e9ac22', 'orderLinkId': ''}, 'retExtInfo': {}, 'time': 1729283916418}
下單成功: Sell 0.001 BTCUSDT
平倉 {'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '4fe86917-607d-49bc-b899-07125ce9